In [ ]:
from scapy.all import *
from scapy.layers.inet import TCP
from scapy.layers.ssl_tls import TLS
from collections import defaultdict
import os

MQTT_STANDARD_PORTS = [1883, 8883]
PUBLISH_PACKET_RANGE = range(0x30, 0x40)

# Output data lists
unencrypted_packets = []
encrypted_packets = []
mqtt_detected_packets = []

# For flood detection per source IP
publish_count_by_ip = defaultdict(int)
FLOOD_THRESHOLD = 50  # Adjust threshold as needed

def is_tls(pkt):
    return pkt.haslayer(TLS)

def get_mqtt_type(pkt):
    if Raw in pkt:
        first_byte = pkt[Raw].load[0]
        return first_byte & 0xF0
    return None

def is_mqtt(pkt):
    """Determine if a packet is an MQTT packet.
    
    The function first checks for the standard MQTT ports. If none of them match,
    it falls back to inspecting the payload for known MQTT control packet signatures.
    """
    if TCP not in pkt:
        return False

    # Check if the packet uses a standard MQTT port
    if pkt[TCP].dport in MQTT_STANDARD_PORTS or pkt[TCP].sport in MQTT_STANDARD_PORTS:
        return True

    # Fallback: Check payload signatures if payload exists.
    if Raw in pkt:
        # Get the first byte of the payload and isolate the MQTT type nibble.
        first_byte = pkt[Raw].load[0]
        mqtt_type = first_byte & 0xF0
        # List all valid MQTT control packet types
        valid_mqtt_types = [
            0x10,  # CONNECT
            0x20,  # CONNACK
            *PUBLISH_PACKET_RANGE,  # PUBLISH (0x30 - 0x3F)
            0x40,  # PUBACK
            0x50,  # PUBREC
            0x60,  # PUBREL
            0x70,  # PUBCOMP
            0x80,  # SUBSCRIBE
            0x90,  # SUBACK
            0xA0,  # UNSUBSCRIBE
            0xB0,  # UNSUBACK
            0xC0,  # PINGREQ
            0xD0,  # PINGRESP
            0xE0,  # DISCONNECT
            0xF0   # AUTH (MQTT 5.0)
        ]
        return mqtt_type in valid_mqtt_types
    return False

def is_mqtt_publish(pkt):
    """Returns True if the packet is a MQTT PUBLISH packet."""
    if Raw in pkt:
        first_byte = pkt[Raw].load[0]
        return (first_byte & 0xF0) in PUBLISH_PACKET_RANGE
    return False

def process_packet(pkt):
    if TCP not in pkt:
        return

    # Check if the packet qualifies as MQTT based on port or payload signature.
    if is_mqtt(pkt):
        src_ip = pkt[IP].src if IP in pkt else "unknown"
        mqtt_detected_packets.append(pkt)

        if is_tls(pkt):
            encrypted_packets.append(pkt)
            print(f"[TLS MQTT] Packet from {src_ip}")
        else:
            unencrypted_packets.append(pkt)
            print(f"[Plain MQTT] Packet from {src_ip}")

        # If packet is MQTT PUBLISH, count the occurrence for flood detection.
        if is_mqtt_publish(pkt):
            publish_count_by_ip[src_ip] += 1
            if publish_count_by_ip[src_ip] > FLOOD_THRESHOLD:
                print(f"[⚠️ ALERT] Possible MQTT PUBLISH flood from {src_ip} ({publish_count_by_ip[src_ip]} packets)")

def main():
    # Use a PCAP file if available, otherwise fallback to live sniffing.
    pcap_file = "mqtt_traffic.pcap"

    if os.path.exists(pcap_file):
        print(f"[*] Reading packets from {pcap_file}")
        packets = rdpcap(pcap_file)
        for pkt in packets:
            process_packet(pkt)
    else:
        print("[*] Sniffing live... (Press Ctrl+C to stop)")
        sniff(prn=process_packet, filter="tcp", store=False)

    # Save categorized MQTT packets to separate files.
    if mqtt_detected_packets:
        wrpcap("mqtt_all_detected.pcap", mqtt_detected_packets)
        print("[*] Saved all MQTT packets to mqtt_all_detected.pcap")
    if encrypted_packets:
        wrpcap("mqtt_encrypted.pcap", encrypted_packets)
        print("[*] Saved encrypted MQTT packets to mqtt_encrypted.pcap")
    if unencrypted_packets:
        wrpcap("mqtt_unencrypted.pcap", unencrypted_packets)
        print("[*] Saved unencrypted MQTT packets to mqtt_unencrypted.pcap")



In [ ]:
from scapy.all import rdpcap, sniff, wrpcap, TCP, IP, Raw, TLS
from collections import defaultdict
import os

# Standard MQTT ports
MQTT_STANDARD_PORTS = {1883: 'unencrypted', 8883: 'encrypted'}
# MQTT Control packet type ranges
PUBLISH_PACKET_RANGE = range(0x30, 0x40)
VALID_MQTT_TYPES = {
    0x10,  # CONNECT
    0x20,  # CONNACK
    *PUBLISH_PACKET_RANGE,  # PUBLISH
    0x40,  # PUBACK
    0x50,  # PUBREC
    0x60,  # PUBREL
    0x70,  # PUBCOMP
    0x80,  # SUBSCRIBE
    0x90,  # SUBACK
    0xA0,  # UNSUBSCRIBE
    0xB0,  # UNSUBACK
    0xC0,  # PINGREQ
    0xD0,  # PINGRESP
    0xE0,  # DISCONNECT
    0xF0   # AUTH (MQTT 5.0)
}

# Flood detection settings
publish_count_by_ip = defaultdict(int)
FLOOD_THRESHOLD = 50  # Adjust as needed

# Packet lists
unencrypted_packets = []
encrypted_packets = []
mqtt_detected_packets = []


def get_mqtt_type(pkt):
    """Extract MQTT control packet type nibble from payload."""
    if Raw in pkt and pkt[Raw].load:
        return pkt[Raw].load[0] & 0xF0
    return None


def categorize_mqtt(pkt):
    """Classify MQTT packet as encrypted or unencrypted according to port and layers."""
    tcp = pkt[TCP]
    src_ip = pkt[IP].src if IP in pkt else 'unknown'

    # 1. Standard port check
    for port, sec in MQTT_STANDARD_PORTS.items():
        if tcp.dport == port or tcp.sport == port:
            return sec

    # 2. Raw layer -> type match
    if Raw in pkt:
        mqtt_type = get_mqtt_type(pkt)
        if mqtt_type in VALID_MQTT_TYPES:
            # Matched MQTT control packet on nonstandard port => unencrypted
            return 'unencrypted'

    # 3. TLS layer fallback
    if TLS in pkt:
        return 'encrypted'

    # no match
    return None


def process_packet(pkt):
    """Process a single packet: detect and categorize MQTT, track floods."""
    if not pkt.haslayer(TCP):
        return

    sec = categorize_mqtt(pkt)
    if sec is None:
        return  # Skip non-MQTT

    # Track all detected MQTT packets
    mqtt_detected_packets.append(pkt)
    src = pkt[IP].src if IP in pkt else 'unknown'

    if sec == 'encrypted':
        encrypted_packets.append(pkt)
        print(f"[TLS MQTT] from {src}")
    else:
        unencrypted_packets.append(pkt)
        print(f"[Plain MQTT] from {src}")

    # Flood detection for PUBLISH messages
    if get_mqtt_type(pkt) in PUBLISH_PACKET_RANGE:
        publish_count_by_ip[src] += 1
        count = publish_count_by_ip[src]
        if count > FLOOD_THRESHOLD:
            print(f"[⚠️ ALERT] MQTT PUBLISH flood from {src}: {count} packets")


def main(pcap_file='mqtt_traffic.pcap'):
    # Read from file or sniff live
    if os.path.exists(pcap_file):
        print(f"[*] Loading {pcap_file}...")
        packets = rdpcap(pcap_file)
        for pkt in packets:
            process_packet(pkt)
    else:
        print("[*] Sniffing live traffic (Ctrl+C to stop)...")
        sniff(prn=process_packet, filter="tcp", store=False)

    # Save outputs
    if mqtt_detected_packets:
        wrpcap("mqtt_all_detected.pcap", mqtt_detected_packets)
        print("[*] Saved all MQTT packets to mqtt_all_detected.pcap")
    if unencrypted_packets:
        wrpcap("mqtt_unencrypted.pcap", unencrypted_packets)
        print("[*] Saved plain MQTT packets to mqtt_unencrypted.pcap")
    if encrypted_packets:
        wrpcap("mqtt_encrypted.pcap", encrypted_packets)
        print("[*] Saved TLS MQTT packets to mqtt_encrypted.pcap")


if __name__ == '__main__':
    main()
